# Mode visualization
In this example, the shear frame mode shapes will be visualized using the spatial plotting tools in KOMA.

In [1]:
from koma import spatial as sp
import numpy as np
import pyvista as pv

from koma.modal import normalize_phi

pv.set_jupyter_backend('trame') # for interactive plots in notebook

## Importing mode shapes
The mode shapes from the eigenvalue solution (reference) are imported. Note that this is done merely out of convenience, these mode shapes could just as easily come directly from an OMA.

In [2]:
# Load mode shapes from eigenvalue solution
omega_n_ref = np.load('./data/omega_n.npy')
xi_ref = np.load('./data/xi_ref.npy')
phi_ref0, __ = normalize_phi(np.load('./data/phi_ref.npy'))

# Flip mode shapes as phi_ref is given with first DOF on top (not on bottom as assumed throughout example)
phi_ref = np.flip(phi_ref0, axis=0)

## Defining system

### Either using classes directly:

In [3]:
# Node(2, 0, 3, 0) creates a node labeled 2 with coordinates (x, y, z) = (0, 3, 0)
nodes = [
    # GROUND FLOOR
    sp.Node(1, 0, 0, 0),
    sp.Node(2, 0, 3, 0),
    sp.Node(3, 3, 0, 0),
    sp.Node(4, 3, 3, 0),
    
    # FIRST FLOOR (1)
    sp.Node(11, 0, 0, 3),
    sp.Node(12, 0, 3, 3),
    sp.Node(13, 3, 0, 3),
    sp.Node(14, 3, 3, 3),
    
    # SECOND FLOOR (2)
    sp.Node(21, 0, 0, 6),
    sp.Node(22, 0, 3, 6),
    sp.Node(23, 3, 0, 6),
    sp.Node(24, 3, 3, 6), 
    
    # THIRD FLOOR (3)
    sp.Node(31, 0, 0, 9),
    sp.Node(32, 0, 3, 9),
    sp.Node(33, 3, 0, 9),
    sp.Node(34, 3, 3, 9), 
    
    # FIFTH FLOOR (4)
    sp.Node(41, 0, 0, 12),
    sp.Node(42, 0, 3, 12),
    sp.Node(43, 3, 0, 12),
    sp.Node(44, 3, 3, 12),
    
    # SIXTH FLOOR (5)
    sp.Node(51, 0, 0, 15),
    sp.Node(52, 0, 3, 15),
    sp.Node(53, 3, 0, 15),
    sp.Node(54, 3, 3, 15),
    
    # WEEKEND ARMS
    sp.Node(4002, 6, 3, 12),
    sp.Node(4004, -3, 3, 12),
    sp.Node(4102, 9, 3, 9),
    sp.Node(4104, -6, 3, 14)
    ]

# Element([1,5], label=2) creates a line element labeled 2 between nodes with labels 1 and 5
# Element([5, 6, 7], label=101) creates a triangle element labeled 101 between nodes 5, 6 and 7
# Note that the label definition is optional, as element labels are not really important for the purpose (node labels are)

E = sp.Element # to make assignments below more compact
elements = [
    E([1,11]), E([2,12]), E([3,13]), E([4,14]),
    E([11,21]), E([12,22]), E([13,23]), E([14,24]),
    E([21,31]), E([22,32]), E([23,33]), E([24,34]),
    E([31,41]), E([32,42]), E([33,43]), E([34,44]),
    E([41,51]), E([42,52]), E([43,53]), E([44,54]),
    
    E([11,12,13]), E([12,13,14]), 
    E([21,22,23]), E([22,23,24]),
    E([31,32,33]), E([32,33,34]),
    E([41,42,43]), E([42,43,44]),
    E([51,52,53]), E([52,53,54]),
    
    E([42, 4002]), E([44, 4004]),
    E([4002, 4102]), E([4004, 4104])
]

model = sp.Model(nodes, elements)
model.plot(background_plotter=False, perspective_cam=False) #background_plotter=False is needed in Jupyter Notebook

Widget(value='<iframe src="http://localhost:61158/index.html?ui=P_0x291e5165fd0_0&reconnect=auto" class="pyvis…

### Or using node matrix and element matrix (more typical setup):

In [3]:
# Nodes
node_matrix = [[1, 0, 0, 0], [2, 0, 3, 0], [3, 3, 0, 0], [4, 3, 3, 0],
               [11, 0, 0, 3], [12, 0, 3, 3], [13, 3, 0, 3], [14, 3, 3, 3],
               [21, 0, 0, 6], [22, 0, 3, 6], [23, 3, 0, 6], [24, 3, 3, 6],
               [31, 0, 0, 9], [32, 0, 3, 9], [33, 3, 0, 9], [34, 3, 3, 9],
               [41, 0, 0, 12], [42, 0, 3, 12], [43, 3, 0, 12], [44, 3, 3, 12],
               [51, 0, 0, 15], [52, 0, 3, 15], [53, 3, 0, 15], [54, 3, 3, 15]]

nodes = sp.nodes_from_matrix(node_matrix)

# Elements
element_matrix = [[1, 1, 11], [2, 2, 12], [3, 3, 13], [4, 4, 14],
                  [11, 11, 21], [12, 12, 22], [13, 13, 23], [14, 14, 24],
                  [21, 21, 31], [22, 22, 32], [23, 23, 33], [24, 24, 34],
                  [31, 31, 41], [32, 32, 42], [33, 33, 43], [34, 34, 44],
                  [41, 41, 51], [42, 42, 52], [43, 43, 53], [44, 44, 54],
                  [101, 11, 12, 13], [102, 12, 13, 14],
                  [201, 21, 22, 23], [202, 22, 23, 24],
                  [301, 31, 32, 33], [302, 32, 33, 34],
                  [401, 41, 42, 43], [402, 42, 43, 44],
                  [501, 51, 52, 53], [502, 52, 53, 54]
                 ]

elements = sp.elements_from_matrix(element_matrix)

model = sp.Model(nodes, elements)
model.plot(background_plotter=False, show=True, node_labels=True, line_settings={'lighting':False, 'color':'black'}, 
           face_settings={'show_edges': True, 'lighting': False})

Widget(value='<iframe src="http://localhost:62178/index.html?ui=P_0x1a76b8d5fd0_0&reconnect=auto" class="pyvis…

## Connecting mode shape to model (the long story)
Now remains the perhaps most complicated definition: to connect the mode shape to our model. This is done by creating a `dofmap` dictionary which maps the degrees of freedom of our displacement (phi) to the degrees of freedom of our model. To do this, we need to create a dictionary with keys defining the node labels of our model and values giving their relationship to either the input displacement or relative to other nodes/DOFs. Also, it is possible to define custom functions to assign a extrapolation/interpolation behaviour.

In our case, we have one-dimensional response data at 5 DOFs (floors) of the shear frame. The other DOFs are defined as zero. Each floor is assumed to move together, so three nodes on each floor are connected to a master node (where our assumed measurement takes place).

Note that DOFs in model not mapped to a displacement DOF is assumed zero by default. For each node in our model we define the connectivity as follows, exemplified by node 11:

    NODE: [disp DOF, disp DOF, disp DOF]   (the dofs refer to the model DOFs)
    
The three displacement DOFs set in our list above define what DOF to assign to each of the three model DOFs of the given node. If we want x and y-components of node 123 to be connected to phi[2] and phi[10] respectively (and z=0), we define this as:

    123: [2, 10, None]
    
Furthermore, you can assign relative connections. These are processed in a secondary step, and refer to the nodes and DOFs of the model (after the initial displacement assignment is conducted). However, both of these steps are given by the same input. To model relative connections (master-slave constraints), you can use the `Rel` object:

    123: [2, 10, Rel(122, 1)]

This adds a secondary connection between the z-component of node 123 (slave) to the y-component (1) of node 122 (master). When all three dofs should be given a master-slave connection to another node, the convenience function `rel3` can be used. This produces output as follows:
    
    rel3(5) = [Rel(3, 0), Rel(3, 1), Rel(3, 2)]
    
    
Finally, the `Rel` object can be initiated either as shown above using node and dof, or by a custom function. To generate the same behaviour os `Rel(122, 1)` using a function we can do:
    
    Rel(fun = lambda n: n(122,1)*1.0)  
    
If we would like to sprinkle it a bit, we can for instance do:
    
    Rel(fun = lambda n: n(122,1)*0.8 - n(121, 2)**0.3)  
    
This implies that our model DOF (wherever we choose to put this Rel-object in our dofmap dictionary) is the added up by the component 1/y of node 122 scaled by 0.8 minus component 2/z of node 121 to the power of 0.3.

Let's assume that our 5 DOFs in the phi matrix correspond to the first model-DOF of nodes 11, 21, 31, 41 and 51.

In [4]:
dofmap = {11: [0, None, None], 
          12: sp.rel3(11),
          13: sp.rel3(11),
          14: sp.rel3(11),
          21: [1, None, None], 
          22: sp.rel3(21),
          23: sp.rel3(21),
          24: sp.rel3(21),
          31: [2, None, None], 
          32: sp.rel3(31),
          33: sp.rel3(31),
          34: sp.rel3(31),
          41: [3, None, None], 
          42: sp.rel3(41),
          43: sp.rel3(41),
          44: sp.rel3(41),
          51: [4, None, None],
          52: sp.rel3(51),
          53: sp.rel3(51),
          54: sp.rel3(51),}                


sensors = {'A1': 11, 'A2': 21, 'A3': 31, 'A4':41, 'A5':51}    # just for plotting
model = sp.Model(nodes, elements, sensors=sensors, dofmap=dofmap)

In [6]:
#%% Plot mode shape mapped on model
model.u = phi_ref[:,1]

pl = model.plot(deformed=False, plot_sensor_nodes=False, plot_nodes=False,
            line_settings={'opacity':0.3}, face_settings={'opacity': 0.3, 'show_edges':False},
                perspective_cam=False, background_plotter=False, show=True)

# model.plot(pl=pl, deformed=True, perspective_cam=False, sensor_labels=True, background_plotter=False, show=True)

C:\Users\kak\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:62178/index.html?ui=P_0x1a76b9d8650_2&reconnect=auto" class="pyvis…

In [15]:
model.get_node(list(sensors.values())[0])


Node 11

To simplify the definition of the dofmap, another (experimental) option is possible. It involves creating an interpolant based on samples from an unstructured grid. It is heavily reliant on a reasonable sensor definition, and is not able to extrapolate. This implies that the nonmeasured DOFs can be constructed based on interpolation between same-component DOFs that are measured.

To accomplish this, the following parameters must be set in the model initialization:

`undefined_dofs`. Defines how undefined DOFs in the `dofmap` are treated. This is set to 'linear', 'nearest' or 'cubic' (methods accepted by `scipy.interpolation.griddata`). When unspecified, the standard option 'zero' is used, and the behaviour seen in the previous cells is resulting. 

`interpolation_axes`. This defines the relevant coordinate components to be used for the interpolation. In our example, this is reasonable to define as [2], as we only want the interpolation field to vary vertically (constant in the xy plane).

Let's use `undefined_dofs`='linear'. Then our `dofmap` can be simplified significantly. We would only need to define our sensor nodes and DOFs (nodes 11, 21, 41 and 51) and our fixed nodes (1,2,3,4), as the rest are based on linear interpolation along the vertical axis:  

In [7]:
dofmap = {11: [0, None, None],
         21: [1, None, None],
         31: [2, None, None],
         41: [3, None, None],
         51: [4, None, None],
          
         1: [None, None, None],
         2: [None, None, None],
         3: [None, None, None],
         4: [None, None, None]
         }

sensors = {'A1': 11, 'A2': 21, 'A3': 31, 'A4':41, 'A5':51}    # just for plotting
model = sp.Model(nodes, elements, sensors=sensors, dofmap=dofmap, undefined_dofs='linear', interpolation_axes=[2])

In [9]:
#%% Plot mode shape mapped on model
model.u = phi_ref[:,4]
pl = model.plot(deformed=False, plot_nodes=False, plot_sensor_nodes=False,
            line_settings={'opacity':0.3}, face_settings={'opacity': 0.3, 'show_edges':False},
                perspective_cam=False, background_plotter=False, show=False)

model.plot(pl=pl, deformed=True, perspective_cam=False, sensor_labels=True, background_plotter=False, show=True)

TypeError: len() of unsized object

## Animate mode shape
Animation of mode shapes can be done in two ways:
 - Interactively in background plotter window (QT)
 - To file (gif or mp4)
 
All inputs to the plot function is also available here. E.g., if no `Plotter()`-object is input using the input `pl`, the views can be set as `view=side`. The appearance of the optional added undeformed reference structure is defined more imprecisely by using common inputs for lines, nodes and faces (implying that keywords must be valid for all these types). For more control, the undeformed reference can be plotted (with `show=False`) to generate the input `Plotter()`-object.
  
### Interactively
When animation is done interactively, the animation speed is not controllable. It can be called as follow (without specifying `filename`):

In [10]:
model.animate_mode(phi_ref[:,2],  node_settings={'color': 'orange'}, line_settings={'color':'red'})

SystemExit: 

C:\Users\kak\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### To file
Alternatively, a file can be generated as follows (gif is shown here). Then, fps (60 is default) and frequency (1 Hz is default) can be specified. 

In [ ]:
for mode in range(3):
    phi_plot = phi_ref[:,mode]/np.max(np.abs(phi_ref[:,mode]))*1.0
    model.animate_mode(phi_plot, fps=60, add_undeformed=True, filename=f'mode{mode+1}.gif')